Compute the FIM of the entire neural network model using autograd functionality in pytorch.

In [1]:
from pathlib import Path
import json
import pickle
from tqdm import tqdm
import subprocess
from multiprocessing import Pool


import numpy as np
import scipy
from scipy.optimize import least_squares
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch

from kliff import nn
from kliff.calculators import CalculatorTorch
from kliff.dataset import Dataset
from kliff.dataset.weight import Weight
from kliff.descriptors import SymmetryFunction
from kliff.loss import Loss
from kliff.models import NeuralNetwork

# Random seed
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

%matplotlib inline
plt.style.use("default")

# Setup

## Variables

In [ ]:
settings = {"partition": "mingjian", "Nlayers": 4, "Nnodes": [128, 128, 128]}

In [2]:
# Read setting file
WORK_DIR = Path().absolute()
ROOT_DIR = WORK_DIR.parent
DATA_DIR = ROOT_DIR / "data"
# with open(ROOT_DIR / "settings.json", "r") as f:
#     settings = json.load(f)
partition = settings["partition"]
suffix = "_".join([str(n) for n in settings["Nnodes"]])
PART_DIR = DATA_DIR / f"{partition}_partition_data"
FP_DIR = PART_DIR / "fingerprints"
RES_DIR = (
    WORK_DIR
    / "results"
    / f"{partition}_partition_{suffix}"
    / "all_parameters_weak_prior"
)
if not RES_DIR.exists():
    RES_DIR.mkdir(parents=True)
JAC_DIR = RES_DIR.parent / "Jacobian" / "test_validation_10%_seed1"
if not JAC_DIR.exists():
    JAC_DIR.mkdir(parents=True)
test_val = "test"

## Model

In [3]:
# Architecture
Nlayers = settings["Nlayers"]  # Number of layers, excluding input layer, including outpt layer
Nnodes = settings["Nnodes"]  # Number of nodes per hidden layer
dropout_ratio = 0.0  # Don't use dropout

In [4]:
# Descriptor
descriptor = SymmetryFunction(
    cut_name="cos", cut_dists={"C-C": 5.0}, hyperparams="set51", normalize=True
)
model = NeuralNetwork(descriptor)

# Layers
hidden_layer_mappings = []
for ii in range(Nlayers - 2):
    hidden_layer_mappings.append(nn.Dropout(dropout_ratio))
    hidden_layer_mappings.append(nn.Linear(Nnodes[ii], Nnodes[ii + 1]))
    hidden_layer_mappings.append(nn.Tanh())

model.add_layers(
    # input layer
    nn.Linear(descriptor.get_size(), Nnodes[0]),  # Mapping from input layer to the first
    nn.Tanh(),  # hidden layer
    # hidden layer(s)
    *hidden_layer_mappings,  # Mappings between hidden layers in the middle
    # hidden layer(s)
    nn.Dropout(dropout_ratio),  # Mapping from the last hidden layer to the output layer
    nn.Linear(Nnodes[-1], 1),
    # output layer
)

# Load best model
orig_model_path = (
    ROOT_DIR
    / "training_dropout"
    / "results"
    / "training"
    / f"{partition}_partition_{suffix}"
    / "model_best_train.pkl"
)
model.load(orig_model_path)

2024-07-14 20:09:23.859 | INFO     | kliff.models.model_torch:load:117 - Model loaded from `/data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/training_dropout/results/training/mingjian_partition_128_128_128/model_best_train.pkl`


## Training set and calculator

In [5]:
# testing set
dataset_path = PART_DIR / "carbon_test_set"
weight = Weight(energy_weight=1.0, forces_weight=np.sqrt(0.1))
tset = Dataset(dataset_path, weight)
configs = tset.get_configs()
nconfigs = len(configs)
print("Total number of configurations:", nconfigs)

# calculator
gpu = False
calc = CalculatorTorch(model, gpu=gpu)
_ = calc.create(
    configs,
    nprocs=20,
    reuse=True,
    fingerprints_filename=FP_DIR / f"fingerprints_test.pkl",
    fingerprints_mean_stdev_filename=FP_DIR / f"fingerprints_test_mean_and_stdev.pkl",
)
bestfit_params = calc.get_opt_params()

2024-07-14 20:09:24.596 | INFO     | kliff.dataset.dataset:_read:398 - 444 configurations read from /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/data/mingjian_partition_data/carbon_test_set
2024-07-14 20:09:24.601 | INFO     | kliff.calculators.calculator_torch:_get_device:592 - Training on cpu
2024-07-14 20:09:24.606 | INFO     | kliff.calculators.calculator_torch:create:93 - Reuse fingerprints `/data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/data/mingjian_partition_data/fingerprints/fingerprints_test.pkl`
2024-07-14 20:09:24.611 | INFO     | kliff.calculators.calculator_torch:create:111 - Reuse fingerprints mean and stdev `/data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/data/mingjian_partition_data/fingerprints/fingerprints_test_mean_and_stdev.pkl`


Total number of configurations: 444


In [6]:
# Loss function
residual_data = {"normalize_by_natoms": True}
loss = Loss(calc, residual_data=residual_data)

# Real calculation

In [7]:
# Utility function
def np_load(filename):
    if Path(filename).exists():
        print("Loading...")
        return np.load(filename)
    else:
        msg = (
            "Please run the companion script (fim_entire_network.py) "
            "to generate the result"
        )
        raise FileNotFoundError(msg)

In [8]:
# Total number of parameters, i.e., weights and biases
nparams = calc.get_num_opt_params()

# Require gradient for all layers
for layer in model.layers:
    layer.requires_grad_(True)

In [9]:
# Each element in the loader only contains 1 configuration
loader = calc.get_compute_arguments(1)
loader_list = list(loader)  # Convert to list to make it easier to index

In [10]:
# Test-validation split
# Load index
with open(JAC_DIR / "test_validation_idx_split.pkl", "rb") as f:
    idx_split = pickle.load(f)
idx_test = idx_split["test"]
idx_val = idx_split["validation"]

loader_list_test = [loader_list[ii] for ii in idx_test]
loader_list_val = [loader_list[ii] for ii in idx_val]

In [11]:
# Compute the minimum cost to be used later in the notebook
C0_times_2_test = loss._get_loss_epoch(loader_list_test)
C0_times_2_val = loss._get_loss_epoch(loader_list_val)

## Reference data

In [12]:
reference_energy = []
energy_natoms = []
for batch in loader:
    batch_energy = np.array([float(sample["energy"]) for sample in batch])
    batch_natoms = np.array([sample["forces"].shape[0] for sample in batch])
    reference_energy = np.append(reference_energy, batch_energy)
    energy_natoms = np.append(energy_natoms, batch_natoms)
print(len(reference_energy))

444


In [13]:
reference_forces = []
forces_natoms = []
for batch in loader:
    batch_forces = [sample["forces"].numpy().flatten() for sample in batch]
    batch_natoms = [
        np.repeat(sample["forces"].shape[0], np.prod(sample["forces"].shape))
        for sample in batch
    ]
    reference_forces = np.append(reference_forces, np.concatenate(batch_forces))
    forces_natoms = np.append(forces_natoms, np.concatenate(batch_natoms))
print(reference_forces.shape)

(68046,)


In [14]:
# Total number of predictions
npreds = len(reference_energy) + len(reference_forces)

## Eigenvalue analysis

**NOTE:**
The eigenvalues are sorted from the smallest to the largest.

In [15]:
# Load the eigenvalues result
eigvals_file = JAC_DIR / f"eighvals_entire_network_{test_val}.npy"
eigvals = np_load(eigvals_file)

Loading...


In [16]:
# # Plot eigenvalues
# plt.figure()
# plt.title("Eigenvalues of the FIM")
# plt.plot(eigvals[::-1])  # Plot the largest to the smallest eigenvalue
# plt.yscale("log")
# plt.ylim(bottom=1e-12)

# # Inset axis
# ax = plt.gca()
# x1, x2, y1, y2 = -50, 1000, 1e-3, 1e6  # subregion of the original image
# axins = ax.inset_axes(
#     [0.5, 0.5, 0.47, 0.47], xlim=(x1, x2), ylim=(y1, y2), yscale="log"
# )
# axins.plot(eigvals[::-1])
# axins.axvline(128, color="k", ls="--")  # There are 129 parameters of the last layer
# ax.indicate_inset_zoom(axins, edgecolor="black")

# plt.show()

In [17]:
# # Count the eigenvalues
# eigvals_lower_bounds_list = np.logspace(-16, 4, 101)
# num_larger_eigvals = [np.sum(eigvals > lb) for lb in eigvals_lower_bounds_list]

# plt.figure()
# plt.title("Number of eigenvalues larger than some cutoff")
# plt.plot(eigvals_lower_bounds_list, num_larger_eigvals)
# plt.xscale("log")
# plt.ylim(bottom=0)
# plt.xlabel("Eigenvalue cutoff")

# # Inset axis
# ax = plt.gca()
# x1, x2, y1, y2 = 1e-3, 1e4, 0, 1000  # subregion of the original image
# axins = ax.inset_axes(
#     [0.5, 0.5, 0.47, 0.47], xlim=(x1, x2), ylim=(y1, y2), xscale="log"
# )
# axins.plot(eigvals_lower_bounds_list, num_larger_eigvals)
# axins.axhline(129, color="k", ls="--")  # There are 129 parameters of the last layer
# ax.indicate_inset_zoom(axins, edgecolor="black")

# plt.show()

Main problem: What is the number of effective parameters?

Some approach:
* In our previous work, we showed that the number of eigenvalues larger than $T_0 = 2 C_0 / N$ (denoted as $n$) is related to the number of effective parameters.
* So, we will find $N$ such that it equals to $n$.
We will turn this into an optimization problem.

In [18]:
# Function to optimize, which computes the difference between N and d
def compute_T(N):
    if test_val == "test":
        return C0_times_2_test / N
    else:
        return C0_times_2_val / N


def find_n_given_N(N):
    T = compute_T(N)
    return sum(eigvals > T)


def find_optimal_N_objective(N):
    n = find_n_given_N(N)
    return N - n


print("Optimization to find the number of effective parameters")
opt = least_squares(find_optimal_N_objective, nparams / 2)
opt_N = int(np.ceil(opt.x))  # Number of effective parameters, round it up
opt_eigvals = eigvals[-opt_N:]  # Eigenvalues of the effective parameters
print("Temperature:", compute_T(opt_N))
opt

Optimization to find the number of effective parameters
Temperature: 0.00022452757001001784


 active_mask: array([0.])
        cost: 0.0
         fun: array([0.])
        grad: array([0.])
         jac: array([[1.]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 9
        njev: 9
  optimality: 0.0
      status: 1
     success: True
           x: array([1215.])

In [19]:
# # Plot eigenvalues
# plt.figure()
# plt.title("Eigenvalues of the FIM")
# plt.plot(eigvals[::-1])  # Plot the largest to the smallest eigenvalue
# plt.axvline(opt_N, c="k", ls="--")
# plt.yscale("log")
# plt.ylim(bottom=1e-12)

# # Inset axis
# ax = plt.gca()
# x1, x2, y1, y2 = -50, 2000, 1e-4, 1e6  # subregion of the original image
# axins = ax.inset_axes(
#     [0.5, 0.5, 0.47, 0.47], xlim=(x1, x2), ylim=(y1, y2), yscale="log"
# )
# axins.plot(eigvals[::-1])
# axins.axvline(opt_N, c="k", ls="--")
# ax.indicate_inset_zoom(axins, edgecolor="black")

# plt.show()

It is interesting that the number of effective parameters we found this way is located near the elbow.
I don't think this is a coincidence.
I have qualitative argument why this behavior is not a coincidence, and it can be shown using the following plot.

In [20]:
# # Look at N vs n
# N = np.arange(nparams / 10) + 1
# with Pool(25) as p:
#     n = np.array(list(p.imap(find_n_given_N, tqdm(N))))

In [21]:
# plt.figure()
# plt.plot(N, N, label="N")
# plt.plot(N, n, label="n")
# plt.legend()
# plt.show()

Some description why we expect the number of effective parameters is around the elbow.

## Eigenvectors analysis

**NOTE:** The eigenvalues are sorted from the smallest to the largest.

In [22]:
# Load the eigenvalues result
eigvecs_file = JAC_DIR / f"eighvecs_entire_network_{test_val}.npy"
eigvecs = np_load(eigvecs_file)

Loading...


In [23]:
# Get the eigenvectors for the effective parameters
opt_eigvecs = eigvecs[:, -opt_N:]
# The following variables are to help plot the eigenvectors
nparams_per_layer = np.append(0, calc.get_size_opt_params()[1])
np.concatenate(
    [[f"Weights layer {n+1}", f"Bias layer {n+1}"] for n in range(settings["Nlayers"])]
)
nparams_per_layer

array([    0,  6528,   128, 16384,   128, 16384,   128,   128,     1])

Note that on the following plots, the left-most column is the sloppiest, and the right-most column is the stiffest

In [24]:
# # Plot participation factor for the relevant parameters. Divide the rows different
# # NN layers. This is to help see which layers are more important.
# for ii in range(len(nparams_per_layer) - 1):
#     start = sum(nparams_per_layer[: ii + 1])
#     end = start + nparams_per_layer[ii + 1]
#     # print(ii, start, end)
#     plt.figure()
#     plt.title(eigvecs_plot_titles[ii])
#     plt.imshow(
#         opt_eigvecs[start:end] ** 2 + 1e-16,
#         aspect="auto",
#         cmap="inferno",
#         norm=mpl.colors.LogNorm(vmin=1e-4, vmax=0.1),
#     )

#     plt.colorbar(label="Participation factor")
#     plt.xlabel(r"Sloppy $\rightarrow$ Stiff")
# plt.show()

# Model ensemble

Now that we know the number of effective parameters, we can compute the overall temperature scaling factor for the likelihood.
Then, the next step is to get the posterior, which we assume to be a Gaussian centered at the optimal parameters.
The covariance of the Gaussian is the inverse of the FIM likelihood plus FIM prior.
Finally, the prior to compute the FIM will be chosen to be a Xavier normal distribution, where the variance for each layer is given by
\begin{equation}
    \sigma^2 = \frac{2}{\text{fan}_{in} + \text{fan}_{out}},
\end{equation}
where $\text{fan}_{in}$ is the number of input features and $\text{fan}_{out}$ is the number of output features.

In the following, we will compute the covariance matrix of the parameters. 
However, note that we will just vary the effective parameters and fix the rest.
We can do this since we know the number of effective parameters and the eigenvectors of the FIM likelihood.

In [25]:
# Construct the FIM of the prior
def compute_xavier_variance(fan_in, fan_out):
    """Compute the Xavier normal variance given number of input and output features."""
    return 2 / (fan_in + fan_out)


# Construct variance for the prior
var_prior = []
for layer in model.layers:
    if isinstance(layer, torch.nn.Linear):
        fan_in = layer.in_features
        fan_out = layer.out_features
        s2 = compute_xavier_variance(fan_in, fan_out)
        var_weight = np.ones(fan_in * fan_out) * s2
        var_bias = np.ones(fan_out)
        var_layer = np.append(var_weight, var_bias)
        var_prior = np.append(var_prior, var_layer)
# Construct the FIM of the prior
scale = 0.1 * eigvals[-opt_N:][0]
fim_prior = scipy.sparse.diags(np.ones_like(var_prior) * scale)

We want to sample from the posterior.
The covariance of the posterior, given that the prior and the likelihood have the same mean, is given by
\begin{equation}
    \mathrm{Cov}_P(\theta) = \left( \mathcal{I}_L(\theta) + \mathcal{I}_\pi(\theta) \right)^{-1}.
\end{equation}

In [26]:
# Parameter samples
p0 = bestfit_params
if test_val == "test":
    C0_times_2 = C0_times_2_test
else:
    C0_times_2 = C0_times_2_val

param_samples_file = RES_DIR / "samples_bayes_parameters.npy"
if param_samples_file.exists():
    print("Loading...")
    param_samples = np.load(param_samples_file)
else:
    # Load FIM likelihood
    fim_file = JAC_DIR / f"fim_entire_network_{test_val}.npy"
    fim_like = np_load(fim_file)
    # Covariance inverse
    # Temperature
    T = C0_times_2 / opt_N
    # Also divide the FIM likelihood by the temperature
    cov_inv = fim_like / T + fim_prior

    # Inverse this matrix to get the covariance in eigenspace
    l, v = np.linalg.eig(cov_inv)
    cov = v @ np.diag(1 / l) @ v.T

    # Samples
    nsamples = 100
    param_samples = np.random.multivariate_normal(mean=p0, cov=cov, size=nsamples)

    # Export the samples
    param_samples_file = RES_DIR / "samples_bayes_parameters.npy"
    np.save(param_samples_file, param_samples)

Loading...


In [27]:
# # Some plots of the samples
# plt.figure()
# plt.plot(np.std(param_samples, axis=0))
# plt.yscale("log")
# plt.xlabel("Parameter index")
# plt.ylabel("Standard deviation")
# plt.show()

In [28]:
# Write and install model ensembles
for ii, params in tqdm(enumerate(param_samples), total=len(param_samples)):
    # Write the model
    complete_params = bestfit_params.copy()
    complete_params[-nparams:] = params
    calc.update_model_params(complete_params)  # Update parameters
    # Install model
    modelpath = RES_DIR / f"{ii:03d}" / f"DUNN_C_fimbayes_{ii:03d}"
    model.write_kim_model(modelpath)  # Write

 23%|██████████████▉                                                  | 23/100 [00:08<00:27,  2.83it/s]2024-07-14 20:11:11.697 | INFO     | kliff.models.neural_network:write_kim_model:121 - KLIFF trained model written to /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/023/DUNN_C_fimbayes_023.


 47%|██████████████████████████████▌                                  | 47/100 [00:16<00:18,  2.85it/s]2024-07-14 20:11:20.135 | INFO     | kliff.models.neural_network:write_kim_model:121 - KLIFF trained model written to /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/047/DUNN_C_fimbayes_047.


 71%|██████████████████████████████████████████████▏                  | 71/100 [00:24<00:10,  2.85it/s]2024-07-14 20:11:28.590 | INFO     | kliff.models.neural_network:write_kim_model:121 - KLIFF trained model written to /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/071/DUNN_C_fimbayes_071.


 95%|█████████████████████████████████████████████████████████████▊   | 95/100 [00:33<00:01,  2.74it/s]2024-07-14 20:11:37.015 | INFO     | kliff.models.neural_network:write_kim_model:121 - KLIFF trained model written to /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/095/DUNN_C_fimbayes_095.


100%|████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.84it/s]


In [29]:
def install_uninstall_model(modelname, mode="install"):
    """This function will install or remove KIM model."""
    kim_command = "kim-api-collections-management"
    if mode == "install":
        flags = ["install", "user"]
    elif mode == "uninstall":
        flags = ["remove", "--force"]
        modelname = Path(modelname).name
    command = np.concatenate(([kim_command], flags, [modelname]))
    subprocess.run(command)


def install_uninstall_wrapper(ii):
    modelname = RES_DIR / f"{ii:03d}" / f"DUNN_C_fimbayes_{ii:03d}"
    # Uninstall first to make sure we don't use the wrong models later
    install_uninstall_model(modelname, "uninstall")
    # Install this model ensemble
    install_uninstall_model(modelname, "install")


with Pool(25) as p:
    p.map(install_uninstall_wrapper, range(100))

This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-

Found local item named: DUNN_C_fimbayes_006.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/006/DUNN_C_fimbayes_006.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_008.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/008/DUNN_C_fimbayes_008.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_019.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/019/DUNN_C_fimbayes_019.
   (If you are trying to install an item from openkim.org
    rerun this comman

-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_001/libkim-api-portable-model.so
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
[done]

Success!

Success!
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_031'. 
Success!
[done]

Success!
[100%] Built target DUNN_C_fimbayes_019
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_032'. [done]

Success!
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
Install the project...
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.


Found local item named: DUNN_C_fimbayes_030.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/030/DUNN_C_fimbayes_030.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_027.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/027/DUNN_C_fimbayes_027.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_029.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/029/DUNN_C_fimbayes_029.
   (If you are trying to install an item from openkim.org
    rerun this comman

Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
Found installed driver... DUNN__MD_292677547454_000
[100%] Built target DUNN_C_fimbayes_026
Install the project...
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_026/libkim-api-portable-model.so

Success!
[100%] Built target DUNN_C_fimbayes_032
[100%] Built target DUNN_C_fimbayes_030
Install the project...
Install the

...

Found local item named: DUNN_C_fimbayes_060.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/060/DUNN_C_fimbayes_060.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_074'. [done]

Success!
Found local item named: DUNN_C_fimbayes_064.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/064/DUNN_C_fimbayes_064.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source direct

This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
[100%] Built target DUNN_C_fimbayes_058
Install the project...

Success!
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_077'. [done]

Success!
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_061/libkim-api-portable-model.so
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.

Success!
Install the project...
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_078'. [done]

Success!
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-5

Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_095'. [done]

Success!
This will remove all files associated with these items.
  Use the '--interactive' option to request confirmation
  before attempting to remove each item.
Removing '/home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_096'. [done]

Success!
Found local item named: DUNN_C_fimbayes_076.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/076/DUNN_C_fimbayes_076.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_075.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/075/DUNN_C_fimbayes_075.
   (If 

Found local item named: DUNN_C_fimbayes_095.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/095/DUNN_C_fimbayes_095.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_094.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/094/DUNN_C_fimbayes_094.
   (If you are trying to install an item from openkim.org
    rerun this command from a different working directory,
    or rename the source directory mentioned above.)

Found local item named: DUNN_C_fimbayes_093.
In source directory: /data/yonatan/myproject/scripts/compare_UQ/neuralnetwork/C/fim/results/mingjian_partition_128_128_128/093/DUNN_C_fimbayes_093.
   (If you are trying to install an item from openkim.org
    rerun this comman

[100%] Built target DUNN_C_fimbayes_082
[100%] Built target DUNN_C_fimbayes_095
Install the project...
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_082/libkim-api-portable-model.so

Success!
Install the project...
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_095/libkim-api-portable-model.so

Success!
[100%] Built target DUNN_C_fimbayes_099
[100%] Built target DUNN_C_fimbayes_093
Install the project...
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimbayes_099/libkim-api-portable-model.so
Install the project...

Success!
-- Install configuration: "None"
-- Installing: /home/byu.local/yonatank/.kim-api/2.2.1+GNU.GNU.GNU.2020-12-28-19-51-27/portable-models-dir/DUNN_C_fimb